In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from sklearn.utils import shuffle



In [2]:
train= pd.read_csv("../input/train.csv")
test= pd.read_csv("../input/test.csv")


In [10]:
feats = ["var_{}".format(i) for i in range(200)]
targets2 = ["target{}".format(i) for i in range(0,2)]

split_tar_df = train.copy(deep=True)
split_tar_df['target1'] = train['target']
split_tar_df['target0'] = 1 - train['target']

zero_subset_df = split_tar_df[split_tar_df['target0'] > 0.9]
ones_subset_df = split_tar_df[split_tar_df['target1'] > 0.9]

frames = [zero_subset_df]
for i in range(0,1):    #best 2
    frames.append(ones_subset_df)

combined_df = pd.concat(frames)
combined_df = shuffle(combined_df)

combined_df.info()

X = combined_df[feats]
X_orig = train.copy(deep=True)[feats]
X_test = test[feats]
y = combined_df[['target1','target0']]

scaler = StandardScaler()

X_sc = scaler.fit_transform(X)
X_test_sc = scaler.fit_transform(X_test)
X_orig_sc = scaler.fit_transform(X_orig)

print(X_sc.shape)
print(y.shape)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 157979 to 146933
Columns: 204 entries, ID_code to target0
dtypes: float64(200), int64(3), object(1)
memory usage: 312.8+ MB
(200000, 200)
(200000, 2)


In [11]:
import keras
from keras.layers import (Flatten, Conv1D, Conv2D, Input, Dense, Dropout, BatchNormalization,
                          concatenate, GaussianNoise, Reshape, TimeDistributed, LeakyReLU, PReLU, Embedding)
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD, Adam
from sklearn.base import BaseEstimator, ClassifierMixin
from pathlib import Path
from keras.callbacks import Callback

class ROC_AUC(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    
    def on_epoch_end(self, epoch, logs={}):
        print("ROC AUC for this fold is ", roc_auc_score(self.y_val, self.model.predict(X_val)))
        
class NNv1(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 inp_shape=200,
                 gaussian_noise=0.07,    #best 0.05
                 dense1_dim=128,
                 dense2_dim=128,
                 dense3_dim=64,
                 dense1_kwargs=None,
                 dense2_kwargs=None,
                 dense3_kwargs=None,
                 classifier_kwargs=None,
                 optimizer=SGD,
                 opt_kwargs=None,
                 ):
        self.inp_shape = inp_shape
        self.gaussian_noise = gaussian_noise
        self.dense1_dim = dense1_dim
        self.dense2_dim = dense2_dim
        self.dense3_dim = dense3_dim
        self.dense1_kwargs = dense1_kwargs
        self.dense2_kwargs = dense2_kwargs
        self.dense3_kwargs = dense3_kwargs
        self.classifier_kwargs = classifier_kwargs
        self.optimizer = optimizer
        self.opt_kwargs = opt_kwargs
        self._default_initiaization()

    def _default_initiaization(self):
        if self.dense1_kwargs is None:
            self.dense1_kwargs = {"kernel_initializer": "glorot_uniform"}
        if self.dense2_kwargs is None:
            self.dense2_kwargs = {"kernel_initializer": "he_uniform"}
        if self.dense3_kwargs is None:
            self.dense3_kwargs = {"kernel_initializer": "he_uniform"}
        if self.classifier_kwargs is None:
            self.classifier_kwargs = {"kernel_initializer": "he_uniform"}
        if self.opt_kwargs is None:
            self.opt_kwargs = {}

    def _build_model(self):
        inp = Input(shape=(self.inp_shape,))
        x = GaussianNoise(self.gaussian_noise)(inp)
        x = Reshape((self.inp_shape, 1))(inp)
        x = Dense(self.dense1_dim, activation='relu',)(x)
        #d1 = TimeDistributed(Dropout(0.2))(d1)
        x = Dense(self.dense1_dim, activation='relu',)(x)
        #d2 = PReLU()(d2)
        #d2 = TimeDistributed(Dropout(0.2))(d2)
        #x = concatenate([d1, d2])
        x = Flatten()(x)
        out = Dense(2, activation='softmax', **self.classifier_kwargs)(x)

        model = Model(inputs=inp, outputs=out)
        opt = self.optimizer(**self.opt_kwargs)
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model

    def fit(self, X, y, *args, **kwargs):
        self.model = self._build_model()
        print(self.model.summary())
        self.model.fit(X, y, *args, **kwargs)
        return self

    def predict(self, X, y=None, weight_path=None, **kwargs):
        if self.model:
            if weight_path is not None:
                self.model.load_weights(weight_path)
            y_hat = self.model.predict(X, **kwargs)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [12]:
y_test_pred = np.zeros(len(test))
    
X_train, X_valid, y_train, y_valid = train_test_split(X_sc, y, test_size=0.2, random_state=42)

model = NNv1(opt_kwargs = {"lr": 0.02, "momentum": 0.9, "nesterov": True, "clipnorm": 1})

model.fit(X_train, y_train, epochs=100, batch_size = 1000, validation_data =(X_valid, y_valid))
pred = model.predict(X_valid)
print(pred.shape)
print( "  auc = ", roc_auc_score(y_valid, pred) )
y_test_pred = model.predict(X_test_sc)[:,0]
y_train_pred = model.predict(X_orig_sc)[:,0]
    

#save base submission
sub_df1 = pd.DataFrame({"ID_code":test["ID_code"].values})
sub_df1["target"] = y_test_pred 
sub_df1.to_csv("submission_full_equal_nn3.csv", index=False)

#save output from train predictions
X_orig["ID_code"]=train["ID_code"]
ens_nn_df = pd.DataFrame({"ID_code":X_orig["ID_code"].values})
ens_nn_df["output"] = y_train_pred
ens_nn_df.to_csv("../input/full_nn_equal_output3.csv")



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200)               0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 200, 1)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 200, 128)          256       
_________________________________________________________________
dense_11 (Dense)             (None, 200, 128)          16512     
_________________________________________________________________
flatten_4 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 51202     
Total params: 67,970
Trainable params: 67,970
Non-trainable params: 0
_________________________________________________________________
None
T

Epoch 51/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1973 - acc: 0.9260 - val_loss: 0.2029 - val_acc: 0.9250
Epoch 52/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1970 - acc: 0.9264 - val_loss: 0.2025 - val_acc: 0.9247
Epoch 53/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1972 - acc: 0.9261 - val_loss: 0.2039 - val_acc: 0.9245
Epoch 54/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1969 - acc: 0.9261 - val_loss: 0.2031 - val_acc: 0.9250
Epoch 55/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1969 - acc: 0.9261 - val_loss: 0.2024 - val_acc: 0.9250
Epoch 56/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1968 - acc: 0.9261 - val_loss: 0.2023 - val_acc: 0.9252
Epoch 57/100
160000/160000 [==============================] - 5s 30us/step - loss: 0.1968 - acc: 0.9263 - val_loss: 0.2028 - val_acc: 0.9246
Epoch 58/100
